# 260. Single Number III

**Medium**

Given an integer array nums, in which exactly two elements appear only once and all the other elements appear exactly twice. Find the two elements that appear only once. You can return the answer in any order.

You must write an algorithm that runs in linear runtime complexity and uses only constant extra space.

# Example 1:

```python
Input: nums = [1,2,1,3,2,5]
Output: [3,5]
```

**Explanation**: [5, 3] is also a valid answer.

# Example 2:

```python
Input: nums = [-1,0]
Output: [-1,0]
```

# Example 3:

```python
Input: nums = [0,1]
Output: [1,0]
```

**Constraints**:

- 2 <= nums.length <= 3 \* 104
- -231 <= nums[i] <= 231 - 1
- Each integer in nums will appear twice, only two integers will appear once.


In [ ]:
"""

### Algorithm: XOR and Bit Partitioning

The core idea is to first use XOR to find the XOR sum of the two unique numbers. Then, we use a distinguishing bit from this XOR sum to partition the original array into two groups. Each group will contain one of the unique numbers and pairs of repeating numbers.

**Detailed Steps:**

1.  **First Pass: Get XOR Sum of the Two Unique Numbers**

      * Initialize `xor_sum = 0`.
      * Iterate through all numbers in `nums`. For each `num`, perform `xor_sum ^= num`.
      * **Theory:** When you XOR all numbers in the array, all elements that appear twice will cancel each other out (because `x ^ x = 0`). The `xor_sum` will thus be equal to `unique1 ^ unique2` (the XOR sum of the two unique numbers). Let's call these unique numbers `a` and `b`. So, `xor_sum = a ^ b`.

2.  **Find a Distinguishing Bit (Rightmost Set Bit)**

      * Since `a` and `b` are different, `a ^ b` (which is `xor_sum`) will be non-zero. This means `xor_sum` must have at least one bit set to 1.
      * This '1' bit indicates a position where `a` and `b` have different values (one has a 0, the other has a 1).
      * We need to find *any* such distinguishing bit. The easiest to find is the rightmost set bit (the least significant bit that is 1) of `xor_sum`.
      * **Technique:** `distinguishing_bit = xor_sum & (-xor_sum)`.
          * **Theory:** In two's complement representation, `-xor_sum` is equivalent to `(~xor_sum + 1)`. When `xor_sum` is ANDed with `-xor_sum`, all bits except the rightmost set bit will be zero. For example, if `xor_sum = 0b110100`, then `-xor_sum` (in a 2's complement system) would effectively be `0b001100` (after considering all bits from rightmost 1 to LSB). `0b110100 & 0b001100 = 0b000100`. This gives us a mask where only the rightmost set bit is 1.

3.  **Second Pass: Partition and XOR**

      * Initialize two variables, `group1_xor_sum = 0` and `group2_xor_sum = 0`.

      * Iterate through all numbers in `nums` again.

      * For each `num`:

          * Check if the `distinguishing_bit` is set in `num` using `(num & distinguishing_bit)`.
          * If `(num & distinguishing_bit) != 0` (i.e., the distinguishing bit is set in `num`):
              * Add `num` to `group1_xor_sum` (perform `group1_xor_sum ^= num`).
          * Else (the distinguishing bit is not set in `num`):
              * Add `num` to `group2_xor_sum` (perform `group2_xor_sum ^= num`).

      * **Theory:** Because `distinguishing_bit` is set for one unique number (`a` or `b`) and not set for the other, it guarantees that `a` and `b` will end up in *different* groups. All pairs of repeating numbers will have the *same* value for `distinguishing_bit` (either both have it set or both don't). Therefore, each pair will fall into the *same* group and will cancel each other out within that group.

      * After this pass, `group1_xor_sum` will be equal to one of the unique numbers (`a` or `b`), and `group2_xor_sum` will be equal to the other.

4.  **Return Result:**

      * Return `[group1_xor_sum, group2_xor_sum]`.

### Example Walkthrough

`nums = [1, 2, 1, 3, 2, 5]`

1.  **First Pass (XOR Sum):**

      * `xor_sum = 1 ^ 2 ^ 1 ^ 3 ^ 2 ^ 5`
      * `xor_sum = (1 ^ 1) ^ (2 ^ 2) ^ 3 ^ 5`
      * `xor_sum = 0 ^ 0 ^ 3 ^ 5`
      * `xor_sum = 3 ^ 5`
      * `3` is `011` in binary.
      * `5` is `101` in binary.
      * `3 ^ 5` is `011 ^ 101 = 110` (which is 6 in decimal).
      * So, `xor_sum = 6`.

2.  **Find Distinguishing Bit:**

      * `xor_sum = 6` (binary `0110`)
      * `distinguishing_bit = xor_sum & (-xor_sum)`
      * `-6` in 2's complement (assuming 8-bit for simplicity) is `11111010`.
      * `00000110 & 11111010 = 00000010` (which is 2 in decimal).
      * So, `distinguishing_bit = 2` (binary `0010`). This bit is the 1st bit (0-indexed).

3.  **Second Pass (Partition and XOR):**

      * `group1_xor_sum = 0`

      * `group2_xor_sum = 0`

      * `distinguishing_bit = 2` (binary `0010`)

      * `num = 1` (0001): `(1 & 2)` is `0`. So, `group2_xor_sum ^= 1` (`0 ^ 1 = 1`).

      * `num = 2` (0010): `(2 & 2)` is `2`. So, `group1_xor_sum ^= 2` (`0 ^ 2 = 2`).

      * `num = 1` (0001): `(1 & 2)` is `0`. So, `group2_xor_sum ^= 1` (`1 ^ 1 = 0`).

      * `num = 3` (0011): `(3 & 2)` is `2`. So, `group1_xor_sum ^= 3` (`2 ^ 3 = 1`).

      * `num = 2` (0010): `(2 & 2)` is `2`. So, `group1_xor_sum ^= 2` (`1 ^ 2 = 3`).

      * `num = 5` (0101): `(5 & 2)` is `0`. So, `group2_xor_sum ^= 5` (`0 ^ 5 = 5`).

      * After loop:

          * `group1_xor_sum = 3`
          * `group2_xor_sum = 5`

4.  **Return:** `[3, 5]` (or `[5, 3]`)

### Complexity Analysis

  * **Time Complexity:** O(N), where N is the number of elements in `nums`. We iterate through the array twice.
  * **Space Complexity:** O(1). We only use a few constant extra variables.

"""


class Solution:
    def singleNumber(self, nums: list[int]) -> list[int]:
        """
        Finds the two elements that appear only once in an array where all other
        elements appear exactly twice.

        Args:
            nums: A list of integers.

        Returns:
            A list containing the two unique elements.
        """

        # Step 1: Get the XOR sum of the two unique numbers (a ^ b)
        # All numbers appearing twice will cancel each other out.
        xor_sum_ab = 0
        for num in nums:
            xor_sum_ab ^= num

        # Step 2: Find a distinguishing bit (rightmost set bit)
        # This bit must be set in either 'a' or 'b', but not both.
        # This allows us to partition the numbers into two groups.
        # xor_sum_ab & (-xor_sum_ab) isolates the rightmost set bit.
        # Example: if xor_sum_ab = 6 (0110), then -6 (two's complement) might be ...11111010.
        # 0110 & 11111010 = 0010 (which is 2)
        distinguishing_bit = xor_sum_ab & (-xor_sum_ab)

        # Step 3: Partition the numbers into two groups based on the distinguishing bit
        # And XOR numbers within each group to find the two unique numbers.
        a = 0  # Will store one unique number
        b = 0  # Will store the other unique number

        for num in nums:
            if (num & distinguishing_bit) != 0:
                # If the distinguishing bit is set in 'num', XOR it into group 'a'
                a ^= num
            else:
                # If the distinguishing bit is NOT set in 'num', XOR it into group 'b'
                b ^= num
        
        # At this point, 'a' will contain one unique number and 'b' will contain the other.
        return [a, b]

# --- Edge Cases and Test Cases ---
if __name__ == "__main__":
    sol = Solution()

    # Example 1: Basic case
    nums1 = [1, 2, 1, 3, 2, 5]
    expected1_options = [[3, 5], [5, 3]]
    result1 = sorted(sol.singleNumber(nums1)) # Sort for consistent comparison
    print(f"Input: {nums1}, Output: {result1}, Expected: {expected1_options[0]} or {expected1_options[1]}")
    assert result1 in expected1_options, f"Test Case 1 Failed: Expected {expected1_options}, Got {result1}"

    # Example 2: Negative numbers
    nums2 = [-1, 0]
    expected2_options = [[-1, 0], [0, -1]]
    result2 = sorted(sol.singleNumber(nums2))
    print(f"Input: {nums2}, Output: {result2}, Expected: {expected2_options[0]} or {expected2_options[1]}")
    assert result2 in expected2_options, f"Test Case 2 Failed: Expected {expected2_options}, Got {result2}"

    # Example 3: Different order, simple numbers
    nums3 = [0, 1]
    expected3_options = [[0, 1], [1, 0]]
    result3 = sorted(sol.singleNumber(nums3))
    print(f"Input: {nums3}, Output: {result3}, Expected: {expected3_options[0]} or {expected3_options[1]}")
    assert result3 in expected3_options, f"Test Case 3 Failed: Expected {expected3_options}, Got {result3}"

    # Edge Case: Larger numbers
    nums4 = [1000, 2000, 1000, 3000, 2000, 5000]
    expected4_options = [[3000, 5000], [5000, 3000]]
    result4 = sorted(sol.singleNumber(nums4))
    print(f"Input: {nums4}, Output: {result4}, Expected: {expected4_options[0]} or {expected4_options[1]}")
    assert result4 in expected4_options, f"Test Case 4 Failed: Expected {expected4_options}, Got {result4}"

    # Edge Case: Mixed positive and negative numbers
    nums5 = [-5, -2, -5, 3, -2, 7]
    expected5_options = [[-3, 7], [7, -3]] # Corrected: -5 ^ -5 = 0, -2 ^ -2 = 0. So 3 ^ 7.
                                         # 3 (0011), 7 (0111). XOR is 0100 (4).
                                         # distinguishing bit is 0100 (4).
                                         # Group1 (bit 2 is set): -2, -2 (cancel), 3. So group1_xor_sum = 3
                                         # Group2 (bit 2 is not set): -5, -5 (cancel), 7. So group2_xor_sum = 7
    result5 = sorted(sol.singleNumber(nums5))
    print(f"Input: {nums5}, Output: {result5}, Expected: {expected5_options[0]} or {expected5_options[1]}")
    assert result5 in expected5_options, f"Test Case 5 Failed: Expected {expected5_options}, Got {result5}"

    # Edge Case: Max/Min integer values (test Python's handling of large numbers)
    max_val = (1 << 31) - 1 # 2147483647
    min_val = -(1 << 31)    # -2147483648
    nums6 = [10, 20, 10, 20, max_val, min_val]
    expected6_options = [[min_val, max_val], [max_val, min_val]]
    result6 = sorted(sol.singleNumber(nums6))
    print(f"Input: {nums6[:4]}..., Output: {result6}, Expected: {expected6_options[0]} or {expected6_options[1]}")
    assert result6 in expected6_options, f"Test Case 6 Failed: Expected {expected6_options}, Got {result6}"

    # Edge Case: Zero as one of the unique numbers
    nums7 = [4, 4, 0, 5]
    expected7_options = [[0, 5], [5, 0]]
    result7 = sorted(sol.singleNumber(nums7))
    print(f"Input: {nums7}, Output: {result7}, Expected: {expected7_options[0]} or {expected7_options[1]}")
    assert result7 in expected7_options, f"Test Case 7 Failed: Expected {expected7_options}, Got {result7}"

    print("\nAll test cases passed!")